In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing.whatifs import scale_up_acc_to_testing

from models.seir import SEIRHD, SEIR_Testing, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.uncertainty import MCUncertainty
from main.seir.optimiser import Optimiser
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from main.seir.sensitivity import gridsearch_single_param, calculate_sensitivity_and_plot

from utils.create_report import create_report
from utils.loss import Loss_Calculator
from utils.enums import Columns
from utils.enums.columns import *

from viz import plot_forecast

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Params

In [ ]:
date_of_interest = '2020-07-31'
forecast_days = 37
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)
ktrials = 10

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, data_from_tracker=True, granular_data=False, filename=None, #Data
    model=SEIRHD, #Choose Model and Ranges
    train_period=21, val_period=30, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased'], #Compartments to Apply Loss on 
    smooth_jump=True)

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']

## Loss DataFrames

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss DataFrame

In [ ]:
predictions_dict['m2']['df_loss']

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['m2']['df_forecast'] = get_forecast(predictions_dict[region], days=forecast_days)
    predictions_dict[region]['m2']['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=False, error_bars=True, days=forecast_days)
    
    predictions, losses, params = get_all_trials(predictions_dict[region], train_fit='m1')
    predictions_dict[region]['m1']['params'] = params
    predictions_dict[region]['m1']['losses'] = losses
    predictions_dict[region]['m1']['predictions'] = predictions
    predictions_dict[region]['m1']['all_trials'] = trials_to_df(predictions, losses, params)
    predictions, losses, params = get_all_trials(predictions_dict[region], train_fit='m2')
    predictions_dict[region]['m2']['params'] = params
    predictions_dict[region]['m2']['losses'] = losses
    predictions_dict[region]['m2']['predictions'] = predictions
    predictions_dict[region]['m2']['all_trials'] = trials_to_df(predictions, losses, params)
    kforecasts = plot_trials(
        predictions_dict[region],
        train_fit='m2',
        predictions=predictions, 
        losses=losses, params=params, 
        k=ktrials,
        which_compartments=[Columns.confirmed, Columns.active])
    predictions_dict[region]['m2']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
    predictions_dict[region]['m2']['forecast_active_topk'] = kforecasts[Columns.active]

## Uncertainty

In [ ]:
region_dict = predictions_dict[('Maharashtra', 'Mumbai')]
uncertainty = MCUncertainty(region_dict, date_of_interest)

In [ ]:
uncertainty.beta

## Create Report


In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region], ROOT_DIR=output_folder)
    predictions_dict[region]['m1']['all_trials'].to_csv(os.path.join(output_folder, 'm1-trials.csv'))
    predictions_dict[region]['m2']['all_trials'].to_csv(os.path.join(output_folder, 'm2-trials.csv'))

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output, filename=os.path.join(output_folder, f'output-{t}.csv'))